# 👨‍🍳 SQS Pizza Shop — Cook Notebook

You are a **cook** at the pizza shop. Your job is to:
1. **Pull orders** from the shared `pizza-orders` queue
2. **See who ordered what** (customer name + toppings)
3. **"Cook" the pizza** (simulated with a short wait)
4. **Post a confirmation** to the `pizza-results` queue so the customer knows their pizza is ready

---

## How This Works (Behind the Scenes)

When you pull an order from the queue, SQS **hides** that message from all other cooks for a period called the **Visibility Timeout**. This prevents two cooks from working on the same pizza.

- If you finish and **delete** the message, it's gone for good — you claimed the order.
- If your notebook crashes or you take too long, the message **reappears** in the queue and another cook can pick it up. This is SQS's built-in fault tolerance.

After "cooking," you post a result to the **response queue** with the same **Correlation ID** that came with the order. This is how the customer's notebook knows which response belongs to them.

---

## Setup: AWS Credentials

If you already have your Colab Secrets configured from Lab 1, just run this cell. Otherwise:
1. Click the **🔑 key icon** in the left sidebar
2. Add `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY`
3. Toggle **Notebook access** ON for each

In [ ]:
!pip install -q boto3

import boto3, os, json, time
from botocore.exceptions import ClientError

# --- Load credentials ---
try:
    from google.colab import userdata
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    print('✅ Loaded credentials from Colab Secrets')
except (ImportError, KeyError):
    print('Not in Colab — using default AWS credential chain')

# --- Verify connection ---
sts = boto3.client('sts', region_name='us-east-1')
identity = sts.get_caller_identity()
print(f"Connected as: {identity['Arn']}")

REGION = 'us-east-1'
sqs = boto3.client('sqs', region_name=REGION)
print('SQS client ready.')

## Step 1: Connect to the Shared Queues

The queue URLs are pre-configured. Just run this cell to verify connectivity.

These are the same two queues that the **Customer** notebooks are using:
- **Order queue**: where customers send orders — you will **read** from here
- **Response queue**: where you post completions — customers will **read** from here

In [ ]:
#  Queue URLs (pre-configured by your instructor)
ORDER_QUEUE_URL = 'https://sqs.us-east-1.amazonaws.com/194722398367/pizza-orders'
RESPONSE_QUEUE_URL = 'https://sqs.us-east-1.amazonaws.com/194722398367/pizza-results'

# Quick sanity check
try:
    attrs = sqs.get_queue_attributes(QueueUrl=ORDER_QUEUE_URL, AttributeNames=['QueueArn'])
    print(f"✅ Order queue connected: {attrs['Attributes']['QueueArn']}")
    attrs = sqs.get_queue_attributes(QueueUrl=RESPONSE_QUEUE_URL, AttributeNames=['QueueArn'])
    print(f"✅ Response queue connected: {attrs['Attributes']['QueueArn']}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Check with your instructor if the queues are set up.")

## Step 2: Enter Your Cook Name

This is the name that will appear in the customer's confirmation. Set it to something recognizable so the customer knows who cooked their pizza!

In [ ]:
COOK_NAME = 'YOUR COOK NAME HERE'  # <-- e.g., 'Chef Dan'

print(f"🧑‍🍳 You are: {COOK_NAME}")
print(f"Ready to cook! Run the next cell to start pulling orders.")

## Step 3: Pull and Cook Orders

This is the heart of the consumer pattern. Here's what happens when you run this cell:

1. **`receive_message()`** asks SQS: "Do you have any orders for me?" It uses **Long Polling** (`WaitTimeSeconds=5`), which means it waits up to 5 seconds for a message before returning empty. This is more efficient than asking every second.

2. When an order arrives, SQS gives it to you and **hides** it from other cooks for the duration of the **Visibility Timeout** (typically 60 seconds). While you're working on it, no other cook can see it.

3. After "cooking" (a short simulated delay), you:
   - **Post a result** to the response queue with the customer's `correlation_id`
   - **Delete the original message** from the order queue (so it doesn't get re-processed)

### Why delete the message?
If you DON'T delete it, SQS assumes you crashed. After the Visibility Timeout expires, the order reappears in the queue and another cook picks it up — resulting in **duplicate pizzas**. The `delete_message()` call is your way of saying "I'm done, throw the ticket away."

### About the ReceiptHandle
When SQS gives you a message, it also gives you a `ReceiptHandle` — a temporary token that proves YOU received this specific copy of the message. You need this token to delete the message. Think of it like a claim ticket at a coat check.

In [ ]:
# --- Pull ONE order at a time ---
# Run this cell each time you want to cook a pizza.
# You can run it multiple times!

print(f"🔍 {COOK_NAME} is looking for orders...\n")

response = sqs.receive_message(
    QueueUrl=ORDER_QUEUE_URL,
    MaxNumberOfMessages=1,
    WaitTimeSeconds=5
)

messages = response.get('Messages', [])

if not messages:
    print("No orders in the queue right now.")
    print("Wait for customers to send orders, then run this cell again.")
else:
    msg = messages[0]
    order = json.loads(msg['Body'])
    
    print(f"📋 ORDER RECEIVED!")
    print(f"   Customer:       {order.get('customer', 'Unknown')}")
    print(f"   Toppings:       {', '.join(order.get('toppings', ['unknown']))}")
    print(f"   Ordered at:     {order.get('ordered_at', '?')}")
    print(f"   Correlation ID: {order.get('correlation_id', 'N/A')[:8]}...")
    
    # Simulate cooking
    print(f"\n   🍕 {COOK_NAME} is cooking...")
    time.sleep(3)
    
    # Post result to the response queue
    result = {
        'correlation_id': order.get('correlation_id'),
        'status': 'READY',
        'cook': COOK_NAME,
        'customer': order.get('customer'),
        'toppings': order.get('toppings'),
        'completed_at': time.strftime('%H:%M:%S')
    }
    
    sqs.send_message(
        QueueUrl=RESPONSE_QUEUE_URL,
        MessageBody=json.dumps(result)
    )
    
    # Delete the order from the order queue
    sqs.delete_message(
        QueueUrl=ORDER_QUEUE_URL,
        ReceiptHandle=msg['ReceiptHandle']
    )
    
    print(f"\n   ✅ DONE! Pizza for {order.get('customer')} is ready.")
    print(f"   Result posted to response queue.")
    print(f"   Original order deleted from order queue.")
    print(f"\n   Run this cell again to cook another pizza!")

## 🎉 Done!

Take a **screenshot** of your output above showing:
- The order(s) you received (customer name + toppings)
- The confirmation(s) you posted back

Upload the screenshot to the **Canvas assignment**.

---

### Want to cook more?
Just keep running the **Step 3** cell. Each run pulls one order. If there are no orders, it means either:
- All orders have been claimed by other cooks
- Customers haven't sent any yet

### Think about this:
- Multiple cooks are all running `receive_message()` on the same queue. SQS guarantees that each message goes to **only one** cook at a time (via the Visibility Timeout).
- If you had 50 cooks and 50 customers, what would happen? What if you had 2 cooks and 50 customers?